In [1]:
import random
import pandas as pd
import numpy as np
import plotly.express as px
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error
import statsmodels.api as sm
import hyperopt as hp
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Time series function
def generate_time_series(start_date, end_date, num_products):
    date_range = pd.date_range(start=start_date, end=end_date, freq="D")
    data = {"date": date_range}

    for i in range(1, num_products + 1):
        product_demand = [random.randint(0, 100) for _ in range(len(date_range))]
        data[f'product_{i}_demand'] = product_demand

    df = pd.DataFrame(data=data)
    
    return df

In [3]:
# Create product demand data
start_date = '2023-01-01'
end_date = '2023-12-31'
num_products = 3

product_demand_data = generate_time_series(start_date, end_date, num_products)
print(product_demand_data)

          date  product_1_demand  product_2_demand  product_3_demand
0   2023-01-01                40                24                69
1   2023-01-02                49                34                93
2   2023-01-03                 4                98                72
3   2023-01-04                36                 3                45
4   2023-01-05                59                97                59
..         ...               ...               ...               ...
360 2023-12-27                83                22                97
361 2023-12-28                74                19                57
362 2023-12-29                98                77                94
363 2023-12-30                77                79                10
364 2023-12-31                15                48                17

[365 rows x 4 columns]


## Rolling Window Cross-Validation

In [4]:
tscv_rolling = TimeSeriesSplit(max_train_size=None, n_splits=5)
folds = []

# Get indices in each fold
for i, (train_index, test_index) in enumerate(tscv_rolling.split(product_demand_data["product_1_demand"])):
    #print("-------------------------------------")
    #print(f"Fold {i + 1}:")
    #print("TRAIN indices:", train_index)
    #print("TEST indices:", test_index)
    #print("")
    folds.append([train_index, test_index])

# Create dfs
folds_df = []
for i in folds:
    df_train = pd.DataFrame(zip(i[0], ['train']*len(i[0])),
                            columns=['index','group'])
    df_test = pd.DataFrame(zip(i[1], ['test']*len(i[1])),
                           columns=['index','group'])
    df = pd.concat([df_train, df_test], axis=0)
    folds_df.append(df)

# Assign the row numbers to each iteration for plotting
list_num = [i[0] for i in list(enumerate(folds))]
list_num.reverse()
list_r = []
for i,j in zip(folds_df, list_num):
    i['k_fold']=[j]*len(i)
    list_r.append(i)
    
df_rolling = pd.concat(list_r, axis=0)
df_rolling.head()


,index,group,k_fold
0,0,train,4
1,1,train,4
2,2,train,4
3,3,train,4
4,4,train,4


In [5]:
fig = px.scatter(df_rolling, x='index', y='k_fold', color='group',
                 color_discrete_map={'test':'red','train':'blue'},
                 title="Cross-Val with rolling window")

fig.update_layout(yaxis_showticklabels=False)
fig.show()

In [6]:
rmse_rolling = []

for i, (train_index, test_index) in enumerate(tscv_rolling.split(product_demand_data["product_1_demand"])):
    X_train, X_test = product_demand_data["product_1_demand"].iloc[train_index], product_demand_data["product_1_demand"].iloc[test_index]
    
    # Fit the model on rolling folds
    arima = sm.tsa.ARIMA(X_train, order=(1, 1, 1)).fit()
    predictions = arima.predict(start=X_test.index.values[0], end= X_test.index.values[-1])

    # Calculate RMSE
    true_values = X_test.values
    rmse_rolling.append(np.sqrt(mean_squared_error(true_values, predictions)))
    print("RMSE Rolling Window - Fold {}: {}".format(i+1, np.mean(rmse_rolling)))


RMSE Rolling Window - Fold 1: 27.866901933121962
RMSE Rolling Window - Fold 2: 28.39120511282475
RMSE Rolling Window - Fold 3: 28.723377486782905
RMSE Rolling Window - Fold 4: 28.57895766063468
RMSE Rolling Window - Fold 5: 28.805679551447405


### Notes
- Cross-validation works under the assumption that the observations are independent. But, this is not true for time series. 
- Rolling Window
- Leakage (It may introduce leakage from future data to the model. The model will observe future patterns to forecast and try to memorize them.)

Sources: 
- https://medium.com/@soumyachess1496/cross-validation-in-time-series-566ae4981ce4
- https://towardsdatascience.com/visualizing-sklearn-cross-validation-k-fold-shuffle-split-and-time-series-split-a13221eb5a56

## Purging

In [7]:
# Set the gap hyperparameter in TimeSeriesSplit
tscv_purging = TimeSeriesSplit(max_train_size=None, n_splits=5, gap = 4) 
purging_folds = []

# Get indices in each fold
for i, (train_index, test_index) in enumerate(tscv_purging.split(product_demand_data["product_1_demand"])):
    #print("-------------------------------------")
    #print(f"Fold {i + 1}:")
    #print("TRAIN indices:", train_index)
    #print("TEST indices:", test_index)
    #print("")
    purging_folds.append([train_index, test_index])

# Create dfs
purging_folds_df = []
for i in purging_folds:
    df_train = pd.DataFrame(zip(i[0], ['train']*len(i[0])),
                            columns=['index','group'])
    df_test = pd.DataFrame(zip(i[1], ['test']*len(i[1])),
                           columns=['index','group'])
    df = pd.concat([df_train, df_test], axis=0)
    purging_folds_df.append(df)

# Assign the row numbers to each iteration for plotting
list_num = [i[0] for i in list(enumerate(folds))]
list_num.reverse()
list_r = []
for i,j in zip(purging_folds_df, list_num):
    i['k_fold']=[j]*len(i)
    list_r.append(i)
    
df_purging = pd.concat(list_r, axis=0)
df_purging.head()

,index,group,k_fold
0,0,train,4
1,1,train,4
2,2,train,4
3,3,train,4
4,4,train,4


In [8]:
fig = px.scatter(df_purging, x='index', y='k_fold', color='group',
                 color_discrete_map={'test':'red','train':'blue'}, title="Cross-Val with Purging")

fig.update_layout(yaxis_showticklabels=False)
fig.show()

In [9]:
rmse_purging = []

for i, (train_index, test_index) in enumerate(tscv_purging.split(product_demand_data["product_1_demand"])):
    X_train, X_test = product_demand_data["product_1_demand"].iloc[train_index], product_demand_data["product_1_demand"].iloc[test_index]
    
    # Fit the model on rolling folds
    arima = sm.tsa.ARIMA(X_train, order=(1, 1, 1)).fit()
    predictions = arima.predict(start=X_test.index.values[0], end= X_test.index.values[-1])

    # Calculate RMSE
    true_values = X_test.values
    rmse_purging.append(np.sqrt(mean_squared_error(true_values, predictions)))
    print("RMSE Purging - Fold {}: {}".format(i+1, np.mean(rmse_purging)))

RMSE Purging - Fold 1: 27.830979761709244
RMSE Purging - Fold 2: 28.377536074592136
RMSE Purging - Fold 3: 28.69972857714753
RMSE Purging - Fold 4: 28.554506602117655
RMSE Purging - Fold 5: 28.78034396853432


### Notes
- The training and validation sets are usually contiguous. 
- Thus, the initial part of the validation set is highly correlated with the last part of the training set.
- Remove the training observations close to the validation set. 
- This increases the independence between training and validation.

Sources:
- https://towardsdatascience.com/4-things-to-do-when-applying-cross-validation-with-time-series-c6a5674ebf3a


## Blocked Cross-Validation

In [10]:
# Create a class to create folds for blocking
class BlockingTimeSeriesSplit():
    def __init__(self, n_splits):
        self.n_splits = n_splits
    
    def get_n_splits(self, X, y, groups):
        return self.n_splits
    
    def split(self, X, y=None, groups=None):
        n_samples = len(X)
        k_fold_size = n_samples // self.n_splits
        indices = np.arange(n_samples)

        margin = 0
        for i in range(self.n_splits):
            start = i * k_fold_size
            stop = start + k_fold_size
            mid = int(0.5 * (stop - start)) + start
            yield indices[start: mid], indices[mid + margin: stop]

In [11]:
tscv_blocking = BlockingTimeSeriesSplit(n_splits=5)
blocking_folds = []

# Get indices in each fold
for i, (train_index, test_index) in enumerate(tscv_blocking.split(product_demand_data["product_1_demand"])):
    #print("-------------------------------------")
    #print(f"Fold {i + 1}:")
    #print("TRAIN indices:", train_index)
    #print("TEST indices:", test_index)
    #print("")
    blocking_folds.append([train_index, test_index])

# Create dfs
blocking_folds_df = []
for i in blocking_folds:
    df_train = pd.DataFrame(zip(i[0], ['train']*len(i[0])),
                            columns=['index','group'])
    df_test = pd.DataFrame(zip(i[1], ['test']*len(i[1])),
                           columns=['index','group'])
    df = pd.concat([df_train, df_test], axis=0)
    blocking_folds_df.append(df)

# Assign the row numbers to each iteration for plotting
list_num = [i[0] for i in list(enumerate(folds))]
list_num.reverse()
list_r = []
for i,j in zip(blocking_folds_df, list_num):
    i['k_fold']=[j]*len(i)
    list_r.append(i)
    
df_blocking = pd.concat(list_r, axis=0)
df_blocking.head()

,index,group,k_fold
0,0,train,4
1,1,train,4
2,2,train,4
3,3,train,4
4,4,train,4


In [12]:
fig = px.scatter(df_blocking, x='index', y='k_fold', color='group',
                 color_discrete_map={'test':'red','train':'blue'}, title="Cross-Val with Blocking")

fig.update_layout(yaxis_showticklabels=False)
fig.show()

In [13]:
rmse_blocking = []

for i, (train_index, test_index) in enumerate(tscv_blocking.split(product_demand_data["product_1_demand"])):
    X_train, X_test = product_demand_data["product_1_demand"].iloc[train_index], product_demand_data["product_1_demand"].iloc[test_index]
    
    # Fit the model on rolling folds
    arima = sm.tsa.ARIMA(X_train, order=(1, 1, 1)).fit()
    predictions = arima.predict(start=X_test.index.values[0], end= X_test.index.values[-1])

    # Calculate RMSE
    true_values = X_test.values
    rmse_blocking.append(np.sqrt(mean_squared_error(true_values, predictions)))
    print("RMSE Blocking - Fold {}: {}".format(i+1, np.mean(rmse_blocking)))

RMSE Blocking - Fold 1: 28.85099349642932
RMSE Blocking - Fold 2: 27.89600899571375
RMSE Blocking - Fold 3: 28.538520442130874
RMSE Blocking - Fold 4: 28.466462163967254
RMSE Blocking - Fold 5: 28.438042430049705


### Notes:
- Avoid leakage from future observations 
- Works by adding margins at two positions. 
- The first is between the training and validation folds in order to prevent the model from observing lag values which are used twice, once as an estimator (regressor) and another as a response. 
- The second is between the folds used at each iteration in order to prevent the model from memorizing patterns from one iteration to the next.

Sources:
- https://goldinlocks.github.io/Time-Series-Cross-Validation/

# Using Hyperopt

In [14]:
# Define a search space
space = {
    'p': hp.hp.randint('p', 11),
    'd': hp.hp.randint('d', 3),
    'q': hp.hp.randint('q', 11)
}


# Define the objective
def objective(params):
    mse_scores = []
    
    tscv = TimeSeriesSplit(n_splits=5)
    for i, (train_index, val_index) in enumerate(tscv.split(product_demand_data["product_1_demand"])):
        X_train, X_val = product_demand_data["product_1_demand"].iloc[train_index], product_demand_data["product_1_demand"].iloc[val_index]
        
        model = sm.tsa.ARIMA(X_train, order=(int(params['p']), int(params['d']), int(params['q'])))
        model_fit = model.fit()
        
        y_pred = model_fit.predict(start=len(X_train), end=len(X_train) + len(X_val) - 1)
        mse = mean_squared_error(X_val, y_pred)
        mse_scores.append(mse)
        #print("Fold {}:".format(i))
        #print("Params per run")
        #print(int(params['p']), int(params['d']), int(params['q']))
        #print("MSE: {}".format(mse))
    
    avg_mse = np.mean(mse_scores)
    return avg_mse # Maximize the average MSE when using hp.fmin

# Configure the Hyperopt search
algo = hp.tpe.suggest  # Tree of Parzen Estimators
max_evals = 100        # Maximum number of iterations

In [15]:
# Run the hyperparameter optimization
best = hp.fmin(fn=objective,
               space=space,
               algo=algo,
               max_evals=max_evals)


# Extract the best hyperparameters
best_p = int(best['p'])
best_d = int(best['d'])
best_q = int(best['q'])



100%|██████████| 100/100 [02:22<00:00,  1.42s/trial, best loss: 827.5182726154923]


In [16]:
# Print best results
print(best)
print("Best parameters are: ")
print(best_p, best_d, best_q)



{'d': 1, 'p': 4, 'q': 3}
Best parameters are: 
4 1 3


### Notes:
- Define a search spcae for your algorithms's hyperparameters
- Define the objective, which shall be optimized
- Configure the Hyperopt search (tell Hyperopt how to find the best hyperparameters)
    - Make sure to check whether the optimization algorithm maximizes or minimizes the objective function (set - if needed before eval metric)
    - hp.fmin minimizes the optimization function respectively
- Run the hyperparameter optimization
- Extract the best hyperparameters
- Fit the model with best parameters (store found hyperparameters in config file for example)

Sources:
- http://hyperopt.github.io/hyperopt-sklearn/#documentation
- https://github.com/hyperopt/hyperopt-sklearn
- https://hyperopt.github.io/hyperopt/